# Topology Validation with GeoFix

Detect spatial errors in geospatial datasets.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AmmarYasser455/GeoFix/blob/main/examples/01_topology_validation.ipynb)

In [ ]:
# !pip install geofix -q

In [ ]:
import geopandas as gpd
import geofix
from shapely.geometry import Polygon, box
from shapely import wkt

## 1. Create a Dataset with Various Errors

We'll create polygons with common geospatial issues:
- **Invalid geometry** (self-intersecting bowtie)
- **Overlapping features**
- **Null geometry**

In [ ]:
# Valid polygon
valid = Polygon([(0, 0), (1, 0), (1, 1), (0, 1)])

# Self-intersecting bowtie (INVALID)
bowtie = Polygon([(0, 0), (2, 2), (2, 0), (0, 2)])

# Overlapping polygon
overlap = Polygon([(0.5, 0.5), (1.5, 0.5), (1.5, 1.5), (0.5, 1.5)])

# Another valid polygon
valid2 = Polygon([(5, 5), (6, 5), (6, 6), (5, 6)])

gdf = gpd.GeoDataFrame(
    {
        "name": ["Valid A", "Bowtie (Invalid)", "Overlaps with A", "Valid B"],
        "type": ["building", "building", "building", "building"],
    },
    geometry=[valid, bowtie, overlap, valid2],
    crs="EPSG:4326",
)

gdf.to_file("mixed_errors.shp")
print(f"Created dataset with {len(gdf)} features")
print(f"Valid geometries: {gdf.geometry.is_valid.sum()}/{len(gdf)}")

## 2. Run Validation

In [ ]:
result = geofix.validate("mixed_errors.shp")
print(result.summary())

## 3. Detailed Error Inspection

In [ ]:
# Inspect invalid geometries
for entry in result['invalid_geometries']:
    print(f"Feature {entry['index']}: {entry['reason']}")

# Inspect overlaps
for overlap in result['overlaps']:
    print(f"Overlap between features {overlap['feature_a']} and {overlap['feature_b']}")
    print(f"  Overlap area: {overlap['overlap_area']:.6f}")

## 4. Quality Score Breakdown

GeoFix computes a 0-100 score based on error rate and CRS presence.

In [ ]:
print(f"Quality Score:   {result['quality_score']}/100")
print(f"Feature Count:   {result['feature_count']}")
print(f"Error Count:     {result['error_count']}")
print(f"CRS:             {result['crs']}")
print(f"\nError Breakdown:")
for etype, count in result['error_breakdown'].items():
    print(f"  {etype}: {count}")